## CASE DE VENDAS

In [1]:
pip install polars

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#### 1) Qual o total de vendas da categoria de refrigerantes no mês de maio de 2025?

In [6]:
import polars as pl

caminho_produtos = r'C:\\Users\\je_hi\\OneDrive\\Área de Trabalho\\SCANNTECH\\BR_PRD_202503_CASE.csv'
caminho_vendas   = r'C:\\Users\\je_hi\\OneDrive\\Área de Trabalho\\SCANNTECH\\BR_VTA_202505_CASE.csv'

# --- Ler produtos como string ---
df_produtos = pl.read_csv(
    caminho_produtos,
    separator=';',
    dtypes={
        "PROD_ID": pl.Utf8,
        "CATEGORIA": pl.Utf8
    }
)

# Normalizar PROD_ID e CATEGORIA
df_produtos = df_produtos.with_columns([
    pl.col("PROD_ID").str.strip_chars().str.to_lowercase(),
    pl.col("CATEGORIA").str.strip_chars().str.to_lowercase()
])

# Filtrar refrigerantes
refri_ids = df_produtos.filter(
    pl.col("CATEGORIA").str.contains("refrig")
)["PROD_ID"].to_list()

print(f"Produtos refrigerantes encontrados: {len(refri_ids)}")

# --- Ler vendas ---
df_vendas = pl.read_csv(
    caminho_vendas,
    separator=';',
    columns=['MES', 'PROD_ID', 'VENDA_VALOR', 'UNIDADES'],
    dtypes={
        "PROD_ID": pl.Utf8,
        "MES": pl.Int64  # MES como número
    }
)

# Normalizar PROD_ID
df_vendas = df_vendas.with_columns([
    pl.col("PROD_ID").str.strip_chars()
])

# Função para converter string BR para float
def str_to_float(col):
    return col.str.replace_all(r'\.', '').str.replace_all(',', '.').cast(pl.Float64)

df_vendas = df_vendas.with_columns([
    str_to_float(pl.col("VENDA_VALOR")).alias("VENDA_VALOR"),
    str_to_float(pl.col("UNIDADES")).alias("UNIDADES")
])

# Filtrar só refrigerantes e mês 202505
df_refri_maio = df_vendas.filter(
    (pl.col("MES") == 202505) &
    (pl.col("PROD_ID").is_in(refri_ids))
)

# Agrupar e somar
resultado = df_refri_maio.select([
    pl.sum("VENDA_VALOR").alias("VENDA_VALOR_TOTAL"),
    pl.sum("UNIDADES").alias("UNIDADES_TOTAL")
])

print("Total de vendas de refrigerantes em maio/2025:")
print(resultado)

# Salvar resultado
resultado.write_csv("resultado_refrigerantes_maio2025.csv")
print("Salvo em resultado_refrigerantes_maio2025.csv")


C:\Users\je_hi\AppData\Local\Temp\ipykernel_9648\4173633998.py:7: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_produtos = pl.read_csv(
C:\Users\je_hi\AppData\Local\Temp\ipykernel_9648\4173633998.py:30: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_vendas = pl.read_csv(


Produtos refrigerantes encontrados: 4500
Total de vendas de refrigerantes em maio/2025:
shape: (1, 2)
┌───────────────────┬────────────────┐
│ VENDA_VALOR_TOTAL ┆ UNIDADES_TOTAL │
│ ---               ┆ ---            │
│ f64               ┆ f64            │
╞═══════════════════╪════════════════╡
│ 7.1059e11         ┆ 1.2919e11      │
└───────────────────┴────────────────┘
Salvo em resultado_refrigerantes_maio2025.csv


#### 2) Qual o total de vendas (valor e unidades) do refrigerante de Cola, lata 269ML, da marca Chilly Recommendation nas lojas da Rede Alfa em Betim?

In [8]:
import polars as pl

# Caminhos
caminho_produtos = r'C:\\Users\\je_hi\\OneDrive\\Área de Trabalho\\SCANNTECH\\BR_PRD_202503_CASE.csv'
caminho_vendas   = r'C:\\Users\\je_hi\\OneDrive\\Área de Trabalho\\SCANNTECH\\BR_VTA_202505_CASE.csv'
caminho_pdv      = r'C:\\Users\\je_hi\\OneDrive\\Área de Trabalho\\SCANNTECH\\BR_PDV_202505_CASE.csv'

# --- Produtos ---
df_produtos = pl.read_csv(
    caminho_produtos,
    separator=';',
    dtypes={
        "PROD_ID": pl.Utf8,
        "CLASIFICACAO_5": pl.Utf8,
        "CLASIFICACAO_9": pl.Utf8,
        "CLASIFICACAO_10": pl.Utf8,
        "MARCA": pl.Utf8,
        "CATEGORIA": pl.Utf8
    }
)

df_produtos = df_produtos.with_columns([
    pl.col("PROD_ID").str.strip_chars().str.to_lowercase(),
    pl.col("CLASIFICACAO_5").str.strip_chars().str.to_lowercase(),
    pl.col("CLASIFICACAO_9").str.strip_chars().str.to_lowercase(),
    pl.col("CLASIFICACAO_10").str.strip_chars().str.to_lowercase(),
    pl.col("MARCA").str.strip_chars().str.to_lowercase(),
    pl.col("CATEGORIA").str.strip_chars().str.to_lowercase()
])

# Filtrar produto específico
refri_ids = df_produtos.filter(
    (pl.col("CATEGORIA").str.contains("refrig")) &
    (pl.col("MARCA").str.contains("chilly recommendation")) &
    (pl.col("CLASIFICACAO_5").str.contains("lata")) &
    (pl.col("CLASIFICACAO_9").str.contains("cola")) &
    (pl.col("CLASIFICACAO_10").str.contains("269ml"))
)["PROD_ID"].to_list()

print(f"Produtos encontrados: {len(refri_ids)}")

# --- PDVs ---
df_pdv = pl.read_csv(
    caminho_pdv,
    separator=';',
    dtypes={
        "PDV_ID": pl.Utf8,
        "REDE": pl.Utf8,
        "LOCAL": pl.Utf8
    }
)

pdv_ids = df_pdv.filter(
    (pl.col("REDE").str.contains("REDE ALFA")) &
    (pl.col("LOCAL").str.contains("BETIM"))
)["PDV_ID"].to_list()

print(f"PDVs encontrados: {len(pdv_ids)}")

# --- Vendas ---
df_vendas = pl.read_csv(
    caminho_vendas,
    separator=';',
    columns=['MES', 'PROD_ID', 'PDV_ID', 'VENDA_VALOR', 'UNIDADES'],
    dtypes={
        "PROD_ID": pl.Utf8,
        "PDV_ID": pl.Utf8,
        "MES": pl.Int64
    }
)

df_vendas = df_vendas.with_columns([
    pl.col("PROD_ID").str.strip_chars(),
    pl.col("PDV_ID").str.strip_chars()
])

# Conversão valores BR
def str_to_float(col):
    return col.str.replace_all(r'\.', '').str.replace_all(',', '.').cast(pl.Float64)

df_vendas = df_vendas.with_columns([
    str_to_float(pl.col("VENDA_VALOR")).alias("VENDA_VALOR"),
    str_to_float(pl.col("UNIDADES")).alias("UNIDADES")
])

# Filtrar produtos e PDVs específicos em maio/2025
df_filtrado = df_vendas.filter(
    (pl.col("MES") == 202505) &
    (pl.col("PROD_ID").is_in(refri_ids)) &
    (pl.col("PDV_ID").is_in(pdv_ids))
)

# Somar valores e unidades
resultado = df_filtrado.select([
    pl.sum("VENDA_VALOR").alias("VENDA_VALOR_TOTAL"),
    pl.sum("UNIDADES").alias("UNIDADES_TOTAL")
])

print("Resultado final:")
print(resultado)

# Salvar CSV
resultado.write_csv("resultado_produto_loja_maio2025.csv")


C:\Users\je_hi\AppData\Local\Temp\ipykernel_9648\409226817.py:9: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_produtos = pl.read_csv(
C:\Users\je_hi\AppData\Local\Temp\ipykernel_9648\409226817.py:43: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_pdv = pl.read_csv(
C:\Users\je_hi\AppData\Local\Temp\ipykernel_9648\409226817.py:61: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_vendas = pl.read_csv(


Produtos encontrados: 1
PDVs encontrados: 87
Resultado final:
shape: (1, 2)
┌───────────────────┬────────────────┐
│ VENDA_VALOR_TOTAL ┆ UNIDADES_TOTAL │
│ ---               ┆ ---            │
│ f64               ┆ f64            │
╞═══════════════════╪════════════════╡
│ 1.5331e6          ┆ 278415.0       │
└───────────────────┴────────────────┘



#### 3) Quais foram os 3 fabricantes com maior faturamento no sudeste e qual tipo de loja teve maior representatividade para cada um deles? 

In [ ]:
import polars as pl

# --- Caminhos dos arquivos ---
caminho_produtos = r'C:\Users\je_hi\OneDrive\Área de Trabalho\SCANNTECH\BR_PRD_202503_CASE.csv'
caminho_vendas   = r'C:\Users\je_hi\OneDrive\Área de Trabalho\SCANNTECH\BR_VTA_202505_CASE.csv'
caminho_pdv      = r'C:\Users\je_hi\OneDrive\Área de Trabalho\SCANNTECH\BR_PDV_202505_CASE.csv'

print("INICIANDO PROCESSAMENTO...")
print("=" * 50)

try:
    # --- Ler produtos ---
    print("Lendo produtos...")
    df_produtos = pl.read_csv(
        caminho_produtos,
        separator=';',
        encoding='utf-8',
        dtypes={
            "PROD_ID": pl.Int64,
            "FABRICANTE": pl.Utf8
        }
    ).with_columns([
        pl.col("FABRICANTE").str.strip_chars().alias("FABRICANTE")
    ])

    # --- Ler PDVs ---
    print("Lendo PDVs...")
    df_pdv = pl.read_csv(
        caminho_pdv,
        separator=';',
        encoding='utf-8',
        dtypes={
            "PDV_ID": pl.Int64,
            "CLASIFICACAO_2": pl.Utf8,
            "TIPO_LOJA": pl.Utf8
        }
    ).with_columns([
        pl.col("CLASIFICACAO_2").str.strip_chars().alias("CLASIFICACAO_2"),
        pl.col("TIPO_LOJA").str.strip_chars().alias("TIPO_LOJA")
    ])

    # Filtrar apenas Sudeste
    df_pdv_sudeste = df_pdv.filter(
        pl.col("CLASIFICACAO_2").str.to_uppercase() == "SUDESTE"
    )
    print(f"PDVs do Sudeste: {df_pdv_sudeste.shape[0]}")

    # --- Ler vendas ---
    print("Lendo vendas...")
    df_vendas = pl.read_csv(
        caminho_vendas,
        separator=';',
        encoding='utf-8',
        columns=['PROD_ID', 'PDV_ID', 'VENDA_VALOR'],
        dtypes={
            "PROD_ID": pl.Int64,
            "PDV_ID": pl.Int64,
            "VENDA_VALOR": pl.Utf8  # Ler como string para converter depois
        }
    )

    print("Convertendo valores...")
    df_vendas = df_vendas.with_columns([
        pl.col("VENDA_VALOR")
        .str.replace_all(r"\.", "")
        .str.replace_all(",", ".")
        .cast(pl.Float64)
        .alias("VENDA_VALOR")
    ])

    # --- Join vendas com produtos para pegar FABRICANTE ---
    print("Fazendo joins...")
    df_join = df_vendas.join(
        df_produtos.select(['PROD_ID', 'FABRICANTE']),
        on='PROD_ID',
        how='left'
    ).join(
        df_pdv_sudeste.select(['PDV_ID', 'TIPO_LOJA']),
        on='PDV_ID',
        how='inner'
    )

    print(f"Registros após join: {df_join.shape[0]}")

    # --- Agrupar por FABRICANTE e somar faturamento ---
    print("Agrupando por fabricante...")
    faturamento_por_fabricante = df_join.group_by("FABRICANTE").agg(
        pl.sum("VENDA_VALOR").alias("FATURAMENTO_TOTAL")
    ).sort("FATURAMENTO_TOTAL", descending=True)

    # --- Selecionar top 3 fabricantes ---
    top3_fabricantes = faturamento_por_fabricante.head(3)["FABRICANTE"].to_list()
    print(f"Top 3 fabricantes: {top3_fabricantes}")

    # --- Para cada fabricante, identificar tipo de loja com maior faturamento ---
    resultados = []
    for fab in top3_fabricantes:
        print(f"Processando fabricante: {fab}")
        tipo_loja_sum = (
            df_join.filter(pl.col("FABRICANTE") == fab)
            .group_by("TIPO_LOJA")
            .agg(pl.sum("VENDA_VALOR").alias("FATURAMENTO_TIPO"))
            .sort("FATURAMENTO_TIPO", descending=True)
        )
        
        if tipo_loja_sum.height > 0:
            top_tipo_loja = tipo_loja_sum[0, "TIPO_LOJA"]
            faturamento_total = faturamento_por_fabricante.filter(
                pl.col("FABRICANTE") == fab
            )["FATURAMENTO_TOTAL"][0]

            resultados.append({
                "FABRICANTE": fab,
                "FATURAMENTO_TOTAL": faturamento_total,
                "TIPO_LOJA_PRINCIPAL": top_tipo_loja
            })

    # --- Criar DataFrame final ---
    df_resultado_final = pl.DataFrame(resultados)
    
    print("\n RESULTADO FINAL:")
    print("=" * 50)
    print(df_resultado_final)

    # --- Salvar resultado ---
    df_resultado_final.write_csv("top3_fabricantes_sudeste.csv")
    print("\n Resultado salvo em 'top3_fabricantes_sudeste.csv'")

except Exception as e:
    print(f" Erro: {e}")
    import traceback
    traceback.print_exc()



INICIANDO PROCESSAMENTO...
Lendo produtos...
Lendo PDVs...


C:\Users\je_hi\AppData\Local\Temp\ipykernel_24424\3486154491.py:14: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_produtos = pl.read_csv(
C:\Users\je_hi\AppData\Local\Temp\ipykernel_24424\3486154491.py:28: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_pdv = pl.read_csv(
C:\Users\je_hi\AppData\Local\Temp\ipykernel_24424\3486154491.py:50: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df_vendas = pl.read_csv(


PDVs do Sudeste: 5134
Lendo vendas...
Convertendo valores...
Fazendo joins...
Registros após join: 13199628
Agrupando por fabricante...
Top 3 fabricantes: ['Rodriguez Group', 'Kuhn and Sons', 'Lynch, Watsica and Predovic']
Processando fabricante: Rodriguez Group
Processando fabricante: Kuhn and Sons
Processando fabricante: Lynch, Watsica and Predovic

 RESULTADO FINAL:
shape: (3, 3)
┌─────────────────────────────┬───────────────────┬─────────────────────┐
│ FABRICANTE                  ┆ FATURAMENTO_TOTAL ┆ TIPO_LOJA_PRINCIPAL │
│ ---                         ┆ ---               ┆ ---                 │
│ str                         ┆ f64               ┆ str                 │
╞═════════════════════════════╪═══════════════════╪═════════════════════╡
│ Rodriguez Group             ┆ 4.7022e9          ┆ Supermercado        │
│ Kuhn and Sons               ┆ 3.3187e9          ┆ Supermercado        │
│ Lynch, Watsica and Predovic ┆ 3.2561e9          ┆ Supermercado        │
└─────────────────────

#### 4) Qual sua sugestão de melhoria na estrutura dos arquivos, visando performance e clareza dos dados?
####
Durante o trabalho realizado tive dificuldades para processar todos os dados da planilha de vendas, que atualmente possui mais de **25 milhões de linhas**.  

#### Substituir CSV por formatos colunares (Parquet/Feather)
- O **CSV** é pesado para leitura, ocupa mais espaço em disco e não possui compressão nativa eficiente.  
- Formatos colunares como **Parquet** ou **Feather** (suportados por **Polars** e **Power BI**) oferecem:
  - Redução significativa do tamanho dos arquivos  
  - Maior velocidade de leitura  
  - Leitura seletiva apenas das colunas necessárias  

Exemplo em **Polars**:
```python
df.write_parquet("vendas.parquet", compression="snappy")

Outro ponto que pode gerar conflito de informações é com relação ao mês que está sendo analisado na tabela de Vendas tem a coluna MES  e na tabela Produto tem a coluna Data_Promoção.

#### 5) Como você faria para estruturar a disponibilização das informações contidas nos arquivos em uma API REST?

In [2]:
import polars as pl
import json

# Caminho do CSV
caminho_csv = r'C:\Users\je_hi\OneDrive\Área de Trabalho\SCANNTECH\BR_PRD_202503_CASE.csv'

# Ler CSV 
df = pl.read_csv(caminho_csv, separator=";")

# Converter para lista de dicionários
lista_dicts = df.to_dicts()

# Salvar em JSON
with open("produtos.json", "w", encoding="utf-8") as f:
    json.dump(lista_dicts, f, ensure_ascii=False, indent=4)

print("Arquivo 'produtos.json' criado com sucesso!")



Arquivo 'produtos.json' criado com sucesso!


Eu faria a disponibilização das informações em uma API REST utilizando Flask (ou outro framework web em Python). A ideia é organizar os dados dos arquivos em endpoints que seguem boas práticas RESTful.

Estrutura do projeto

main.py: código principal da API

produtos.json: armazenamento dos dados (simulando um banco de dados)

requirements.txt: dependências

Procfile: configuração para deploy (Railway/Heroku)

Rotas principais da API

GET /produtos → retorna todos os produtos contidos no arquivo produtos.json

POST /produtos → recebe um JSON no corpo da requisição e adiciona um novo produto no arquivo

(opcionalmente poderíamos ter GET /produtos/<id>, PUT /produtos/<id>, DELETE /produtos/<id> para operações completas de CRUD)

Formato de dados
As respostas são retornadas em JSON, que é o padrão para APIs REST.
Exemplo de resposta:

[
    {"id": 1, "nome": "Arroz", "preco": 12.50},
    {"id": 2, "nome": "Feijão", "preco": 8.90}
]


Deploy em nuvem

Usei o Railway para hospedar a API.

O servidor é iniciado com gunicorn main:app (onde main é o nome do arquivo e app a aplicação Flask).

A API ficou disponível publicamente em uma URL, permitindo o consumo remoto.

Link da API: https://web-production-ff7fe.up.railway.app/produtos